In [1]:
import os 
import os.path as osp
os.chdir(osp.dirname(os.getcwd()))
os.getcwd()

'/home/DISCOVER_summer2022/xusc/exp/butd_detr'

In [2]:
from loguru import logger
from my_script.pc_utils import *
from my_script.utils import *
from src.join_dataset import JointDataset

from src.join_unlabeled_dataset import JointUnlabeledDataset
from src.join_labeled_dataset import JointLabeledDataset




Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


WARNING - 2022-10-23 15:59:58,097 - collision - No FCL -- collision checking will not work
WARNING - 2022-10-23 15:59:58,203 - svg_io - SVG path loading unavailable!


In [3]:
train_dataset = JointDataset(
            # dataset_dict={'sr3d':1,'scannet':10},
            dataset_dict={'scannet':10},
            test_dataset='sr3d', #? only test set need ? 
            split='train',
            use_color=True, use_height=False,
            overfit=True,
            data_path='datasets/',
            detect_intermediate=True,#? 
            use_multiview=False, #? 
            butd=False,
            butd_gt=False,
            butd_cls=True,
            augment_det=False
        )



Loading train files, take a breath!


2022-10-23 15:57:57.852 | INFO     | src.join_dataset:load_annos:178 -  scannet : 128  annotations   loaded 


## unlabeled datasets

In [3]:
train_dataset = JointUnlabeledDataset(
            # dataset_dict={'sr3d':1,'scannet':10},
            dataset_dict={'scannet':10},
            test_dataset='sr3d', #? only test set need ? 
            split='train',
            use_color=True, use_height=False,
            overfit=True, #* debug 
            data_path='datasets/',
            detect_intermediate=True,#? 
            use_multiview=False, #? 
            butd=False,
            butd_gt=False,
            butd_cls=True,
            augment_det=False,
        )

Loading train files, take a breath!


2022-10-23 15:54:38.576 | INFO     | src.join_unlabeled_dataset:load_annos:164 - 1199  annotations   loaded 


## labeled datasets

In [3]:

train_dataset = JointLabeledDataset(
            # dataset_dict={'sr3d':1,'scannet':10},
            dataset_dict={'scannet':10},
            test_dataset='sr3d', #? only test set need ? 
            split='train',
            use_color=True, use_height=False,
            overfit=True,
            data_path='datasets/',
            detect_intermediate=True,#? 
            use_multiview=False, #? 
            butd=False,
            butd_gt=False,
            butd_cls=True,
            augment_det=False
        )

Loading train files, take a breath!


2022-10-23 16:00:29.847 | INFO     | src.join_labeled_dataset:load_annos:165 - 1199  annotations   loaded 


In [4]:


def save_(data,scene_name,save_root,has_color= True,flag = "debug"):
      #* for teacher or student 
      
    make_dirs(osp.join(save_root,scene_name))

    if has_color:
        write_pc_as_ply(
                    data['point_clouds'],
                    os.path.join(save_root,scene_name, '%s_gt_%s.ply'%(scene_name,flag))
                )
    else :
        write_ply(
            data['point_clouds'],
            os.path.join(save_root,scene_name, '%s_gt_%s.ply'%(scene_name,flag))
        )

        

    #* open3d  format bounding box 
    # np.savetxt(os.path.join(save_root, scene_name,'%s_box_%s.txt'%(scene_name,flag)),
    #         data['all_bboxes'][data['all_bbox_label_mask']],fmt='%s')
    np.savetxt(os.path.join(save_root, scene_name,'%s_box_%s.txt'%(scene_name,flag)),
            data['all_detected_boxes'][data['all_detected_bbox_label_mask']],fmt='%s')


def save_teacher(data,scene_name,save_root,has_color= True,flag = "debug"):
      #* for teacher or student 
      
    make_dirs(osp.join(save_root,scene_name))

    if has_color:
        write_pc_as_ply(
                    data['pc_before_aug'],
                    os.path.join(save_root,scene_name, '%s_gt_%s.ply'%(scene_name,flag))
                )
    else :
        write_ply(
            data['pc_before_aug'],
            os.path.join(save_root,scene_name, '%s_gt_%s.ply'%(scene_name,flag))
        )

        

    #* open3d  format bounding box 
    # np.savetxt(os.path.join(save_root, scene_name,'%s_box_%s.txt'%(scene_name,flag)),
    #         data['teacher_box'][data['all_bbox_label_mask']],fmt='%s')
    np.savetxt(os.path.join(save_root, scene_name,'%s_box_%s.txt'%(scene_name,flag)),
            data['teacher_box'][data['all_detected_bbox_label_mask']],fmt='%s')




debug_path = "logs/debug"
make_dirs(debug_path)
for  idx, data in enumerate(train_dataset):
    save_(data,data['scan_ids'],debug_path,flag='student')
    save_teacher(data,data['scan_ids'],debug_path,flag='teacher')
    logger.info(f"data['scan_ids']:{data['scan_ids']}")
    if idx ==3:
        break


2022-10-23 16:00:34.954 | INFO     | __main__:<module>:58 - data['scan_ids']:scene0191_00
2022-10-23 16:00:39.460 | INFO     | __main__:<module>:58 - data['scan_ids']:scene0191_01
2022-10-23 16:00:43.848 | INFO     | __main__:<module>:58 - data['scan_ids']:scene0191_02
2022-10-23 16:00:48.278 | INFO     | __main__:<module>:58 - data['scan_ids']:scene0119_00


In [7]:
for  idx, data in enumerate(train_dataset):
    print_attr_shape(data)
    
    if idx ==2:
        break

2022-10-22 21:11:17.569 | INFO     | my_script.utils:print_attr_shape:313 - box_label_mask : (132,)
2022-10-22 21:11:17.573 | INFO     | my_script.utils:print_attr_shape:313 - center_label : (132, 3)
2022-10-22 21:11:17.574 | INFO     | my_script.utils:print_attr_shape:313 - sem_cls_label : (132,)
2022-10-22 21:11:17.574 | INFO     | my_script.utils:print_attr_shape:313 - size_gts : (132, 3)
2022-10-22 21:11:17.575 | INFO     | my_script.utils:print_attr_shape:295 - scan_ids : scene0191_00
2022-10-22 21:11:17.576 | INFO     | my_script.utils:print_attr_shape:313 - point_clouds : (50000, 6)
2022-10-22 21:11:17.576 | INFO     | my_script.utils:print_attr_shape:295 - utterances : cabinet . bed . chair . couch . table . door . window . bookshelf . picture . counter . desk . curtain . refrigerator . shower curtain . toilet . sink . bathtub . other furniture . not mentioned
2022-10-22 21:11:17.577 | INFO     | my_script.utils:print_attr_shape:313 - tokens_positive : (132, 2)
2022-10-22 21:11

In [5]:

prompt_demo_idx = 128
train_dataset.__getitem__(prompt_demo_idx)['utterances']
get_attr(data)
print_attr_shape(data)
print(f" box num : {(data['box_label_mask']==1).sum()}")
print(f" label number :  {len(data['utterances'].split('.'))}")
#todo : label 如何映射的? 
#todo :  utterances : 如何设计的 ?  如何设计positive and negtive
#* 


2022-10-22 21:08:12.409 | INFO     | my_script.utils:print_attr_shape:313 - box_label_mask : (132,)
2022-10-22 21:08:12.411 | INFO     | my_script.utils:print_attr_shape:313 - center_label : (132, 3)
2022-10-22 21:08:12.413 | INFO     | my_script.utils:print_attr_shape:313 - sem_cls_label : (132,)
2022-10-22 21:08:12.415 | INFO     | my_script.utils:print_attr_shape:313 - size_gts : (132, 3)
2022-10-22 21:08:12.416 | INFO     | my_script.utils:print_attr_shape:295 - scan_ids : scene0528_00
2022-10-22 21:08:12.416 | INFO     | my_script.utils:print_attr_shape:313 - point_clouds : (50000, 6)
2022-10-22 21:08:12.417 | INFO     | my_script.utils:print_attr_shape:295 - utterances : cabinet . bed . chair . couch . table . door . window . bookshelf . picture . counter . desk . curtain . refrigerator . shower curtain . toilet . sink . bathtub . other furniture . not mentioned
2022-10-22 21:08:12.418 | INFO     | my_script.utils:print_attr_shape:313 - tokens_positive : (132, 2)
2022-10-22 21:08

 box num : 8
 label number :  19


In [16]:


def get_caption(utterances):
    caption = ' '.join(utterances.replace(',', ' ,').split())
    caption = ' ' + caption + ' '

    
    return caption
'''
description: 根据tokens_positive 从utterances 取text token 
param {*} utterances
param {*} tokens_positive
return {*}
'''
def get_token(utterances,tokens_positive):

    caption = get_caption(utterances)
    all_text_token_from_utterance = []
    for start,end in tokens_positive:
        tmp = caption[start:end+1].strip()
        all_text_token_from_utterance.append(tmp)
        # print(tmp,end='\t')
    return all_text_token_from_utterance


logger.info(data['utterances'])
all_labels = data['utterances'].split('.')
logger.info(f" utterances length : {len(all_labels)}")

# all_labels
#?  negtive label 为什么会有box 与之对应呢?    多出来的两个box 是什么?  应该是与negtive sample对应的... 
all_box = data['all_bboxes'][data['all_bbox_label_mask']]
logger.info(f" box  number in the scene  : {len(all_box)}")
all_target_box = data['center_label'][data['box_label_mask']==1]
logger.info(f"referred tagret box number  in the scene  : {all_target_box.shape[0]}") 


logger.info(f"referred target object sem_cls_label: {data['sem_cls_label'][data['box_label_mask']==1]}")#* 这个没用吗? 为什么sem_cls_label 都是对应0 ? 

# logger.info(f"data['positive_map'].shape {data['positive_map'].shape}")#* 这个跟 language model 的输出有关
all_text_token_from_potive_token = get_token(data['utterances'],data['tokens_positive'][data['box_label_mask']==1])

text_from_utterance_appear_in_positive_token = []
for label in all_labels:
    if label.strip() in all_text_token_from_potive_token:
    
        text_from_utterance_appear_in_positive_token.append(label)
logger.info(f"  {len(text_from_utterance_appear_in_positive_token)} object in the utterance is referred ")
logger.info(f"all text token referred as postive token  {all_text_token_from_potive_token}")
logger.info(f" {len(all_text_token_from_potive_token)} referred as postive token")
#* 用utterances 去找只能找到  10个, 而referred target有15个, 也就是说有5个重复的目标, 


logger.info(f"data['distractor_ids'] : {data['distractor_ids']},length : {len(data['distractor_ids'])}")
logger.info(f",data['anchor_ids'] : {data['anchor_ids']},len : {len( data['anchor_ids'])}")
logger.info(f"data['target_cid'] : {data['target_cid']}")





2022-10-21 14:52:20.817 | INFO     | __main__:<module>:24 - cabinet . bed . chair . couch . table . door . window . bookshelf . picture . counter . desk . curtain . refrigerator . shower curtain . toilet . sink . bathtub . other furniture . not mentioned
2022-10-21 14:52:20.819 | INFO     | __main__:<module>:26 -  utterances length : 19
2022-10-21 14:52:20.820 | INFO     | __main__:<module>:31 -  box  number in the scene  : 17
2022-10-21 14:52:20.821 | INFO     | __main__:<module>:33 - referred tagret box number  in the scene  : 7
2022-10-21 14:52:20.822 | INFO     | __main__:<module>:36 - referred target object sem_cls_label: [ 5  4  6 17  2  6  6]
2022-10-21 14:52:20.824 | INFO     | __main__:<module>:46 -   5 object in the utterance is referred 
2022-10-21 14:52:20.825 | INFO     | __main__:<module>:47 - all text token referred as postive token  ['door', 'table', 'window', 'other furniture', 'chair', 'window', 'window']
2022-10-21 14:52:20.825 | INFO     | __main__:<module>:48 -  7 

In [8]:
data['all_bboxes'][data['all_bbox_label_mask']].shape

(17, 6)

In [9]:
data['all_class_ids'][data['all_bbox_label_mask']]

array([  0,   0,   4,   3,  62,  62,  14,  42,  50,   1,  14, 325,  46,
        14,  46,   2,   7])